In [1]:
from utils import *

In [2]:
blog_author_regex = "<wp:author_login><!\[CDATA\[(.*?)\]\]>.*<wp:author_display_name><!\[CDATA\[(.*?)]]><\/wp:author_display_name>.*<\/wp:author>"
mag_author_regex = "<!\[CDATA\[_issuem_author_name\]\]><\/wp:meta_key>[\s\S]*?<wp:meta_value><!\[CDATA\[(.*?)\]\]"

media_regex = "<item>[\s\S]*?<guid .*?>(.*?)<\/guid>[\s\S]*?<wp:post_parent>(.*?)<\/wp:post_parent>[\s\S]*?<\/item>"
blog_regex = "<item>[\s\S]*?<title>(.*?)<\/title>[\s\S]*?<link>(.*?)<\/link>[\s\S]*?<dc:creator><!\[CDATA\[(.*?)\]\]><\/dc:creator>[\s\S]*?p=(.*?)<\/guid>[\s\S]*?<content:encoded><!\[CDATA\[([\s\S]*?)\]\][\s\S]*?<wp:post_date><!\[CDATA\[(.*?)\]\]><\/wp:post_date>[\s\S]*?<\/item>"
mag_regex = '<item>[\s\S]*?<title>(.*?)<\/title>[\s\S]*?<link>(.*?)<\/link>[\s\S]*?<guid[\s\S]*?p=(.*?)<\/guid>[\s\S]*?<content:encoded><!\[CDATA\[([\s\S]*?)\]\]><\/content:encoded>[\s\S]*?<wp:post_date><!\[CDATA\[(.*?)\]\]><\/wp:post_date>[\s\S]*?<category domain="issuem_issue"[\s\S]*?<!\[CDATA\[(.*?)\]\][\s\S]*?<!\[CDATA\[_issuem_author_name\]\]><\/wp:meta_key>[\s\S]*?<wp:meta_value><!\[CDATA\[(.*?)\]\][\s\S]*?<\/item>'

authors = {}

with open('posts/authors.xml', 'r') as xml:
    contents = xml.read()
    for i, auth_display_name in re.findall(blog_author_regex, contents):
        authors[i] = auth_display_name

with open('posts/magazine-articles.xml', 'r') as xml:
    contents = xml.read()
    for auth_display_name in re.findall(mag_author_regex, contents):
        authors[auth_display_name] = auth_display_name

media = {}
        
with open('posts/media.xml', 'r') as xml:
    contents = xml.read()
    for url, parent_id in re.findall(media_regex, contents):
        media[parent_id] = url        
        
authors_df = pd.DataFrame(authors.values()).replace('', np.nan).dropna()
article_title_categories = pd.read_csv('posts/article-categories.csv')
categories = pd.read_csv('posts/categories.csv', header=None)

In [3]:
blog_posts = []

with open('posts/export.xml', 'r') as xml:
    contents = xml.read()
    draw_progress_bar(0)
    matches = re.findall(blog_regex, contents)
    for i, post in enumerate(matches):
        blog_posts.append({})
        ID = post[3]

        blog_posts[-1]["id"] = ID
        blog_posts[-1]["bsr_permalink"] = post[1]
        blog_posts[-1]["title"] = markdownify(post[0])
        blog_posts[-1]["author"] = authors[post[2]]
        
        
        if post[5]:
            blog_posts[-1]["created_at"] = parser.parse(post[5]).strftime('%d %B %Y %I:%M%p')
            
        blog_posts[-1]['issue'] = np.nan
        
        if ID in media:
            blog_posts[-1]['featured_image'] = media[ID]
            flag = 'featured_image given'
            
        else:
            blog_posts[-1]['featured_image'], flag = retrieve_img(post[1])
            
        newlines = re.sub('\n', '<br><br>', post[4])
        hlines = re.sub('<hr />', '-------', newlines)
        out = re.sub('<!-- .*? -->', '', hlines)
        final = re.sub(r'\[caption .*?\](.*?\/>) (.*?)\[\/caption\]', r'\1<br><em>\2</em>', out)

        md = markdownify(final, heading_style=ATX).lstrip()
        final = re.sub('\n\s*\n', '<br><br>', md)
#         final = re.sub(r'\[caption .*?src="(.*?)"[\s\S]*?\[\/caption\]',r'<img src="\1"/>',final)
#         print(final)
        
        blog_posts[-1]["markdown"] = markdownify(final, heading_style=ATX).lstrip()

        draw_progress_bar(i/len(matches), flag)

blog_posts = pd.DataFrame(blog_posts)
blog_posts = blog_posts[~blog_posts['bsr_permalink'].str.contains("trashed")]

[======================================= ] 100%       featured_image given not subscriptablend'

In [4]:
articles = []

with open('posts/magazine-articles.xml', 'r') as xml:
    contents = xml.read() 
    draw_progress_bar(0)
    matches = re.findall(mag_regex, contents)
    for i, article in enumerate(matches):
        articles.append({})
        ID = article[2]

        articles[-1]["id"] = ID
        
        articles[-1]["bsr_permalink"] = article[1]
        articles[-1]["title"] = markdownify(article[0])
        articles[-1]["author"] = authors[article[6]]
        
        if article[5]:
            articles[-1]["created_at"] = parser.parse(article[4]).strftime('%d %B %Y %I:%M%p')

        articles[-1]['issue'] = article[5]
            
        if ID in media:
            articles[-1]['featured_image'] = media[ID]
            flag = 'featured_image given'
        else:
            articles[-1]['featured_image'], flag = retrieve_img(article[1])
            
        newlines = re.sub('\n', '<br><br>', article[3])
        hlines = re.sub('<hr />', '-------', newlines)
        out = re.sub('<!-- .*? -->', '', hlines)
        final = re.sub(r'\[caption .*?\].*?(<img.*?\/>).*?<\/a> .*?(.*?)\[\/caption\]', r'\1<br><em>\2</em>', out)
        final = re.sub(r'\[caption .*?\](.*?\/>) (.*?)\[\/caption\]', r'\1<br><em>\2</em>', final)
        

        md = markdownify(final, heading_style=ATX).lstrip()
        final = re.sub('\n\s*\n', '<br><br>', md)
        
        articles[-1]["markdown"] = markdownify(final, heading_style=ATX).lstrip()

        draw_progress_bar(i/len(matches), flag)

articles = pd.DataFrame(articles)
articles = articles[~articles['bsr_permalink'].str.contains("trashed")]

[======================================= ] 100%       'NoneType' object is not subscriptablend'

In [6]:
merged = pd.concat([blog_posts,articles],axis=0)

In [7]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


def fuzzy_merge(df_1, df_2, key1, key2, threshold=95, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    c = df_1['matches'].apply(lambda x: ', '.join([str(i[1]) for i in x]))
    df_1['matches'] = m2
    df_1['confidence'] = c

    return df_1

In [8]:
articles_w_categories_fuzzy = fuzzy_merge(article_title_categories, merged, 'title', 'title', threshold=90)

In [9]:
articles_w_categories_fuzzy = articles_w_categories_fuzzy[articles_w_categories_fuzzy.confidence.astype(int)>95]
fuzzy = articles_w_categories_fuzzy.drop(['title','confidence'], axis=1).rename(columns={"matches": "title"})

In [10]:
final_article_df = merged.merge(fuzzy, on='title', how='left')

In [11]:
final_article_df.describe()

,id,bsr_permalink,title,author,created_at,issue,featured_image,markdown,primary_category,secondary_category
count,1127,1127,1127,1127,1127,369,804,1127,214,73
unique,1125,1125,1095,276,1043,22,799,1116,11,11
top,16850,https://berkeleysciencereview.com/article/from...,From the Editor,Psych Your Mind,10 May 2020 09:03PM,Fall 2019,https://berkeleysciencereview.com/wp-content/u...,,Life Science,Life Science
freq,2,2,12,112,17,24,2,8,53,25


In [13]:
articles.loc[367, 'bsr_permalink']

'https://berkeleysciencereview.com/article/from-the-editor-fall-2020/'

In [18]:
final_article_df.to_json('export-2020-12-26.js', orient='records', indent=1)